## This notebook inputs the missing values using KNN

In [3]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [4]:
INPUT, OUTPUT = 'output/CompleteDengue.csv', 'output/\KNNDengueImputed'

In [10]:
df = pd.read_csv(r'output\/CompleteDengue.csv',index_col=0)
df_train = df.loc['1990-04-30':'2006-07-16'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2006-07-23':, df_train.columns] #Selecting test data
df = df_train

In [11]:
df.shape

(844, 40)

In [9]:
df.head(845)

,sj_ndvi_ne,sj_ndvi_nw,sj_ndvi_se,sj_ndvi_sw,sj_precipitation_amt_mm,sj_reanalysis_air_temp_k,sj_reanalysis_avg_temp_k,sj_reanalysis_dew_point_temp_k,sj_reanalysis_max_air_temp_k,sj_reanalysis_min_air_temp_k,...,iq_reanalysis_precip_amt_kg_per_m2,iq_reanalysis_relative_humidity_percent,iq_reanalysis_sat_precip_amt_mm,iq_reanalysis_specific_humidity_g_per_kg,iq_reanalysis_tdtr_k,iq_station_avg_temp_c,iq_station_diur_temp_rng_c,iq_station_max_temp_c,iq_station_min_temp_c,iq_station_precip_mm
1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,299.764286,295.851429,302.4,298.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,299.221429,295.865714,301.3,297.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,299.528571,296.531429,300.6,298.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,299.557143,296.378571,302.1,297.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,300.278571,296.651429,302.3,298.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
for i in range(0,30):
    #Scaled data has zero mean and unit variance
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array_test = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
    
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    imp.fit(df_scaled)
    df_imputed = imp.transform(df_scaled)
    df_test_imputed = imp.transform(df_test_scaled)
    df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
    df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
    # Descaling the data
    array_2 = scaler.inverse_transform(df_final)
    df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

    array_test_2 = scaler.inverse_transform(df_test_final)
    df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
    df_final2.to_csv(OUTPUT+"_"+str(i)+'_training.csv', index = True)
    df_test_final2.to_csv(OUTPUT+"_"+str(i)+'_test.csv', index = True)